Sales Prediction for Big Mart Outlets
The data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities. Also, certain attributes of each product and store have been defined. The aim is to build a predictive model and predict the sales of each product at a particular outlet.

Using this model, BigMart will try to understand the properties of products and outlets which play a key role in increasing sales.

Please note that the data may have missing values as some stores might not report all the data due to technical glitches. Hence, it will be required to treat them accordingly. 

Data Dictionary
We have train (8523) and test (5681) data set, train data set has both input and output variable(s). You need to predict the sales for test data set.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns #importing seaborn module 
import warnings
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler 
from sklearn import metrics
from pandas import set_option
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold, cross_val_score # to split the data
from sklearn.metrics import explained_variance_score, median_absolute_error, r2_score, mean_squared_error, accuracy_score, confusion_matrix, classification_report, fbeta_score #To evaluate our model
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split # Model evaluation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler # Preprocessing
from sklearn.linear_model import Lasso, Ridge, ElasticNet, RANSACRegressor, SGDRegressor, HuberRegressor, BayesianRidge # Linear models
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor  # Ensemble methods
from xgboost import XGBRegressor, plot_importance # XGBoost
from sklearn.svm import SVR, SVC, LinearSVC  # Support Vector Regression
from sklearn.tree import DecisionTreeRegressor # Decision Tree Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline # Streaming pipelines
from sklearn.decomposition import KernelPCA, PCA # Dimensionality reduction
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import learning_curve, validation_curve, GridSearchCV # Model evaluation
from sklearn.base import clone # Clone estimator
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import xgboost as xgb
warnings.filterwarnings('ignore')

In [ ]:
#Load train and test files 
data = pd.read_csv('../input/big-mart-sales-prediction/Train.csv')
test = pd.read_csv('../input/big-mart-sales-prediction/Test.csv')

In [ ]:
#creating dataframe for the required output for submission file

submission = pd.DataFrame()
submission['Item_Identifier'] = test['Item_Identifier']
submission['Outlet_Identifier'] = test['Outlet_Identifier']

#Step 1 : Explore train and test datasets

In [ ]:

#First look at train
data.sample(5)

In [ ]:
#First look at test
test.sample(5)

In [ ]:
#Shape of train and test
print('There are {} rows and {} columns in train'.format(data.shape[0],data.shape[1]))
print('There are {} rows and {} columns in train'.format(test.shape[0],test.shape[1]))

In [ ]:
#Check Missing values in train
data.isna().sum()

In [ ]:
#Check Missing values in test
test.isna().sum()

In train & test - Item_Weight & Outlet_Size have missing values

In [ ]:
#check datatypes in train
data.info()

In [ ]:
#check datatypes in test
test.info()

In [ ]:
#Lets describe train
data.describe()

In [ ]:
#Lets describe test
test.describe()

In [ ]:
#Lets concatenate train & test
df=pd.concat([data,test])
df.shape 

Step 2 : EDA

In [ ]:
#Lets convert Year from numerical to object type 
df['Outlet_Establishment_Year']  = df['Outlet_Establishment_Year'].astype('object')

In [ ]:

df.Item_Identifier.nunique()
#df['Item_Identifier'].value_counts().sample(10)

There are 1559 unique values of Item_Identifier with max frequency of 10 and min is 7

In [ ]:
#Identify categorical columns 
df_cat=df.select_dtypes(include='object')
df_cat.drop(['Item_Identifier'],axis=1,inplace=True)
df_cat.columns

In [ ]:
#Identify numerical columns
df_num=df.select_dtypes(include=['int64','float64'])
list(df_num.columns)

In [ ]:
#Explore categorical variables - 'Item_Fat_Content', 'Outlet_Size' 
cat_col_1 = ['Item_Fat_Content', 'Outlet_Size']
count = 1
for cols in cat_col_1:
    plt.subplot(2, 2, count)
    df[cols].value_counts().plot.pie(shadow=True,autopct='%1.1f%%',radius=1.1,textprops={'fontsize': 9} )
    count +=1
    plt.subplot(2, 2, count)
    plt.tight_layout()
    plt.style.use('fivethirtyeight')
    df[cols].value_counts().plot.bar()
    fig=plt.gcf()
    fig.set_size_inches(12,7)
    plt.title('{0}'.format(cols))
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=5)
    plt.xticks(rotation=30)
    count+=1

* Item fat content : Low Fat is having maximum (59.7%) count
* outlet size      : Medium size outlet (45.7%) is having max count

In [ ]:
#Explore categorical variables - 
cat_col_1 = [ 'Item_Type', 'Outlet_Identifier']
count = 1
for cols in cat_col_1:
    plt.subplot(2, 2, count)
    df[cols].value_counts().plot.pie(shadow=True,autopct='%1.1f%%',radius=1.2,textprops={'fontsize': 8} )
    count +=1
    plt.subplot(2, 2, count)
    plt.tight_layout()
    plt.style.use('fivethirtyeight')
    df[cols].value_counts().plot.bar()
    fig=plt.gcf()
    fig.set_size_inches(18,9)
    plt.title('{0}'.format(cols))
    plt.xticks(fontsize=8)
    plt.yticks(fontsize=9)
    plt.xticks(rotation=30)
    count+=1

* Item type: Fruits and vegetables has max count of 2k and seafood is min
* outlet identifier : outl027 is having max countand out019 is having min count

In [ ]:
#Explore categorical variables - 
cat_col_1 = [ 'Outlet_Location_Type', 'Outlet_Type']
count = 1
for cols in cat_col_1:
    plt.subplot(2, 2, count)
    df[cols].value_counts().plot.pie(shadow=True,autopct='%1.1f%%',radius=1.0,textprops={'fontsize': 9} )
    count +=1
    plt.subplot(2, 2, count)
    plt.tight_layout()
    plt.style.use('fivethirtyeight')
    df[cols].value_counts().plot.bar()
    fig=plt.gcf()
    fig.set_size_inches(13,8)
    plt.title('{0}'.format(cols))
    plt.xticks(fontsize=8)
    plt.yticks(fontsize=9)
    plt.xticks(rotation=30)
    count+=1

* Outlet_Location_Type : Max count for Tier3 (39.3%) and Tier 1 is having lowest count
* Outlet_Type : Supermarket Type1 has mx count (65.4%)

In [ ]:
#Explore categorical variables - 
cat_col_1 = [ 'Outlet_Establishment_Year']
count = 1
for cols in cat_col_1:
    plt.subplot(1, 2, count)
    df[cols].value_counts().plot.pie(shadow=True,autopct='%1.1f%%',radius=1.2,textprops={'fontsize': 10} )
    count +=1
    plt.subplot(1, 2, count)
    plt.tight_layout()
    plt.style.use('fivethirtyeight')
    df[cols].value_counts().plot.bar()
    fig=plt.gcf()
    fig.set_size_inches(12,4)
    plt.title('{0}'.format(cols))
    plt.xticks(fontsize=8)
    plt.yticks(fontsize=9)
    plt.xticks(rotation=30)
    count+=1

* 1985: Maximum count of 2300+ can be seen
* 1998 : Minimum count of 900 can be seen
* All other years have same count which is surprising

In [ ]:
sns.set(style='whitegrid', palette="plasma", font_scale=1.1, rc={"figure.figsize": [8, 5]})
df[list(df_num.columns)].hist(bins=15, figsize=(15, 6), layout=(2, 4));

* Item Weight     : We can see maximum frequency  between 6-20
* Sales           : Max occurances for sales value 0-2k, few sales can be seen around 9k as well 
* Item_MRP        : Price frequency is max between 100-140 & 150-190
* Item_Visibility : Maximum frequency between 0.01-.05

In [ ]:
#Numerical Data ('Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales' Vs Outlet_Type)
plt.figure(figsize=(10, 14))
count = 1
for cols in df_num:
    sns.set(style='darkgrid', palette="Set1", font_scale=1.1, rc={"figure.figsize": [8, 6]})
    plt.subplot(4, 2, count)
    plt.tight_layout()
    #sns.boxenplot(x='Outlet_Type', y= cols, data=data)
    sorted_nb = df.groupby(['Outlet_Type'])[cols].median().sort_values()
    sns.boxenplot(x=df['Outlet_Type'], y=df[cols], order=list(sorted_nb.index))
    plt.xticks(rotation=30)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=9)
    
    count+=1

* Item weight : Its same across diff outlets. Item weight is missing for super mkt 3 and grocery as per below stats
* Item_Visibility : Grocery stores have highest visibility, rest all super markets have same visibility
* Item_MRP : MRP is same across all outlets which is obvious
* Item_Outlet_Sales : Median Sales volume are highest at super mkt type 3 > super mkt 1 >  super mkt 2 > grocery

In [ ]:
#Numerical Data ('Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales' Vs Item_Fat_Content)
plt.figure(figsize=(10, 14))
count = 1
for cols in df_num:
    sns.set(style='whitegrid', palette="rocket", font_scale=1.1, rc={"figure.figsize": [8, 6]})
    plt.subplot(4, 2, count)
    plt.tight_layout()
    #sns.boxenplot(x='Outlet_Type', y= cols, data=data)
    sorted_nb = df.groupby(['Item_Fat_Content'])[cols].median().sort_values()
    sns.boxenplot(x=df['Item_Fat_Content'], y=df[cols], order=list(sorted_nb.index))
    plt.xticks(rotation=30)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=9)
    
    count+=1

* Item weight :  "low fat" is having min median item weight < LF < Regular < reg < Low Fat
* Item_Visibility : same across all categories
* Item_MRP : Lf < reg & low fat < Low Fat < Regular
* Item_Outlet_Sales : Median Sales volume reg < low fat < Low Fat < LF < Regular (But not much variance among diff categories)

In [ ]:
#Numerical Data ('Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales' Vs Item_Type)
plt.figure(figsize=(14, 12))
count = 1
for cols in df_num:
    sns.set(style='darkgrid', palette="ocean", font_scale=1.1)
    plt.subplot(4, 2, count)
    plt.tight_layout()
    #sns.boxenplot(x='Outlet_Type', y= cols, data=data)
    sorted_nb = df.groupby(['Item_Type'])[cols].median().sort_values()
    sns.boxenplot(x=df['Item_Type'], y=df[cols], order=list(sorted_nb.index))
    plt.xticks(rotation=60)
    plt.xticks(fontsize=9)
    plt.yticks(fontsize=9)
    
    count+=1

* Item weight :  hard drinks is having min median item weight & others having max median value
* Item_Visibility : meat is lowest & breakfast is highest in median values
* Item_MRP : baking goods is lowest and starchy foods is highest in median values 
* Item_Outlet_Sales : soft drinks have min median Vs sea food is having maximum median sales value 

In [ ]:
#Numerical Data ('Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales' Vs Outlet_Location_Type)
plt.figure(figsize=(14, 12))
count = 1
for cols in df_num:
    sns.set(style='ticks', palette="tab10", font_scale=1.1)
    plt.subplot(4, 2, count)
    plt.tight_layout()
    #sns.boxenplot(x='Outlet_Type', y= cols, data=data)
    sorted_nb = df.groupby(['Outlet_Location_Type'])[cols].median().sort_values()
    sns.boxenplot(x=df['Outlet_Location_Type'], y=df[cols], order=list(sorted_nb.index))
    plt.xticks(rotation=60)
    plt.xticks(fontsize=9)
    plt.yticks(fontsize=9)
    
    count+=1

* Item weight :  same across all 
* Item_Visibility : Tier 2 < 3 & 1
* Item_MRP : same across all
* Item_Outlet_Sales : Tier 1 < 3 < 2 (i.e. Tier 2 generates highest sales median value)

In [ ]:
#Numerical Data ('Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales' Vs Outlet_Size)
plt.figure(figsize=(14, 12))
count = 1
for cols in df_num:
    sns.set(style='dark', palette="rocket", font_scale=1.1)
    plt.subplot(4, 2, count)
    plt.tight_layout()
    #sns.boxenplot(x='Outlet_Type', y= cols, data=data)
    sorted_nb = df.groupby(['Outlet_Size'])[cols].median().sort_values()
    sns.boxplot(x=df['Outlet_Size'], y=df[cols], order=list(sorted_nb.index))
    plt.xticks(rotation=60)
    plt.xticks(fontsize=9)
    plt.yticks(fontsize=9)
    
    count+=1

* Item weight :  same across all 
* Item_Visibility : medium  < high & small
* Item_MRP : same across all
* Item_Outlet_Sales : small < high < medium (i.e. medium generates highest sales median value)

In [ ]:
#Numerical Data ('Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales' Vs Outlet_Establishment_Year)
plt.figure(figsize=(14, 12))
count = 1
for cols in df_num:
    sns.set(style='whitegrid', palette="rocket", font_scale=1.1)
    plt.subplot(4, 2, count)
    plt.tight_layout()
    #sns.boxenplot(x='Outlet_Type', y= cols, data=data)
    sorted_nb = df.groupby(['Outlet_Establishment_Year'])[cols].median().sort_values()
    sns.boxenplot(x=df['Outlet_Establishment_Year'], y=df[cols], order=list(sorted_nb.index))
    plt.xticks(rotation=60)
    plt.xticks(fontsize=9)
    plt.yticks(fontsize=9)
    
    count+=1

* Item weight :  same across all (1985 has misising data)
* Item_Visibility : 1999 had lowest Vs 1998 had highest
* Item_MRP : almost same across all
* Item_Outlet_Sales : 1998 had lowest sales Vs 1987&2004 had highest (i.e. 1987 & 2004 generates highest sales median value)

Multivariate Analysis

In [ ]:
#Variable - Outlet_Type Vs Item_Outlet_Sales
#Catplot
sns.catplot(x='Outlet_Location_Type',y='Item_Outlet_Sales',kind='boxen',data=df,col='Outlet_Type',hue='Outlet_Size',palette="Set1")

Tier 3 in type super mkt 3 is having highest median sales

Imputing missing values

In [ ]:
#Item Weight - Item weight should depend upon the item type and Item_Fat_Content a per above numerical analysis with boxenplots
Item_Weight_group = df.groupby(["Item_Type","Item_Fat_Content"])["Item_Weight"]

#printing the variabe that we created by median
print(Item_Weight_group.median())

df = df.reset_index()

In [ ]:
# using the groupby to transform this variables
df.loc[df.Item_Weight.isnull(), 'Item_Weight'] = df.groupby(['Item_Type','Item_Fat_Content']).Item_Weight.transform('median')

In [ ]:
# printing the total of nulls in Item_Weight Feature
print(df["Item_Weight"].isnull().sum())

Imputing missing values : Outlet_Size

In [ ]:
#Identify outlet type & Outlet_Location_Type for missing Outlet_Type
print(df[df.Outlet_Type.isnull()].Outlet_Size.value_counts())

In [ ]:
df.loc[df.Outlet_Size.isnull(),['Outlet_Type','Outlet_Location_Type']].sample(5)

* Imputing missing values for Outlet Size :
* There are three types of outlet sizes : small/medium/large
* The mussing values exist for the below combination of outlet type & outlet location type
    * a. Grocery & Tier 3
    * b. Super market 1 & Tier 2

* As per the above boxen plot we can see that for
    * a. outlet type = grocery generally the outlet size is small 
    * b. outlet type = super market generally the outlet size is small & medium.
* Hence I decided to imput small value for all missing values of Outlet Size

In [ ]:
df['Outlet_Size'].fillna('Small', inplace=True)


In [ ]:
df.isnull().sum()

All null values are imputed and data is ready for modelling

In [ ]:
#Reset index
df.drop('index', axis = 1, inplace = True)
#Drop non-imp columns
df.drop('Item_Identifier', axis = 1, inplace = True)

In [ ]:
#Converting categorocal variables to dummy variables 
df=pd.get_dummies(df,drop_first=True)

# Segregating train and test from df
train=df[:data.shape[0]]
test1=df[data.shape[0]:]

In [ ]:
#Shape of train and test
print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in test'.format(test1.shape[0],test1.shape[1]))

train.drop('Item_Outlet_Sales', axis = 1, inplace = True)
test1.drop('Item_Outlet_Sales', axis = 1, inplace = True)

In [ ]:
validation_size = 0.20
seed = 7

In [ ]:
# Split-out validation dataset
X_train, X_validation, Y_train, Y_validation = train_test_split(train.values, data['Item_Outlet_Sales'].values, test_size=validation_size, random_state=seed)

After trying out many models, I decided to use GBM to predict, as it was showing lowest RMSE

In [ ]:
#Standard scaling
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
param_grid = dict(n_estimators=np.array([80,82,84,85,90,91,92,94,95,150,200]))

In [ ]:
#Use GridSearchCV to find best estimator & RMS (root mean square)to calculate score
num_folds = 10
RMS = 'neg_mean_squared_error'
model = GradientBoostingRegressor(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=RMS, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#Make predictions on validation dataset using best estimstor = 80
# prepare the model
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model = GradientBoostingRegressor(alpha=0.9,learning_rate=0.05, max_depth=5, min_samples_leaf=3, min_samples_split=2, n_estimators=80, random_state=30)
model.fit(rescaledX, Y_train)
# transform the validation dataset
rescaledValidationX = scaler.transform(X_validation)
predictions = model.predict(rescaledValidationX)
print("RMSE = ", mean_squared_error(Y_validation, predictions))

Scale the test dataset

In [ ]:
#Stanrdard scaling of the test dataset
rescaled_test1 = scaler.transform(test1)

Prepare the submission file

In [ ]:
#Predict on the test dataset
predicted_prices = model.predict(rescaled_test1)

#PRepare submission file
submission['Item_Outlet_Sales'] = predicted_prices
submission.to_csv('./submission_rahulpednekar.csv',index=False)
submission.head()

> ****This prediction achieved 106th position (RMSE : 1144) in Analytics Vidhya Hackathon on Big Mart Sales  

> > **If you have liked my Kernel then please UPVOTE**